<div align="right">Python 2.7 Jupyter Notebook</div>

# Data Exploration: Test your intuition about BSSIDs

### Your completion of the notebook exercises will be graded based on your ability to: 

> **Understand**: Does your pseudo-code and/or comments show evidence that you recall and understand technical concepts?

> **Apply**: Are you able to execute code, using the supplied examples, that perform the required functionality on supplied or generated data sets? 

> **Evaluate**: Are you able to interpret the results and justify your interpretation based on the observed data?

# Notebook introduction

In this notebook you will examine the WiFi scan dataset that you loaded as a dataset in one of the exercises in Module 1, Notebook 2. You will also use the public dataset from Dartmouth College, [Student Life](http://studentlife.cs.dartmouth.edu/dataset.html), that was introduced in Module 1.

Before continuing with this notebook, it is important to understand the definition of a [BSSID](http://www.juniper.net/techpubs/en_US/network-director1.1/topics/concept/wireless-ssid-bssid-essid.html). A BSSID (Basic Service Set Identifier) is the  media access control (MAC) address (or physical address) of a wireless access point (WAP). It is generated by combining the 24-bit Organization Unique Identifier (the manufacturer's identity) and the manufacturer's assigned 24-bit identifier for the radio chipset in the WAP. In short, every router has a unique address, which will be utilized in this notebook.

In an analysis, you will start with an idea which you need to validate. In Video 2 of this module, Arek Stopczynski suggests that you should try to test ideas on yourself first, as this is the easiest way of validating your assumptions about the data generated. As a result of this, you will be able to quickly spot anomalies based on your understanding of your own behavior and patterns. Once you have a functional dataset and hypothesis, you should also start to consider cases where the behaviors of others do not necessarily align to your own.

Manual functions are used to review the data in many cases. When performing an analysis, you need to validate all your assumptions and be able to logically describe what you want to do before selecting a method of execution. In some cases functions utilized may behave in unexpected ways, and you therefore need to constantly perform checks to ensure that the output values are correct and as expected. Pandas is a tried and tested library with a large following of loyal users, however, some of the other libraries that you come across may not be as well-tested.

> **Note**: 

> It is strongly recommended that you save a checkpoint after applying significant changes or completing exercises. This allows you to return the notebook to a previous state should you wish to do so. On the Jupyter menu, select "File", then "Save and Checkpoint" from the dropdown menu that appears.

#### Load libraries

In [1]:
from os import path
import pandas as pd

## 1. Dataset exploration

#### 1.1 Load data
To start the process, load a single user's data. For this example you will start with the first recordset, u00.

In [2]:
# Load the data for a specific user
dataset_path = '../data/dartmouth/wifi/'
user00 = pd.read_csv(path.join(dataset_path, 'wifi_u00.csv'))

In [3]:
# Display the head.
user00.head(5)

,time,BSSID,freq,level
0,1364356963,00:60:b3:07:1d:27,2427,-90
1,1364356963,f8:1e:df:fd:4a:4b,2417,-64
2,1364358163,f8:1e:df:fd:4a:4b,2417,-69
3,1364358320,f8:1e:df:fd:4a:4b,2417,-74
4,1364359364,f8:1e:df:fd:4a:4b,2417,-62


#### 1.2 Review data definitions (if any)

The table below provides some definitions for the dataset, which can aid you in better understanding the data.

Each row represents a WiFi access point seen by a user’s phone. There are four columns in the the provided dataset:

| Column  | Description |
| ------------- | ------------- |
| time | Timestamp of the observation (epochtime format). |
| BSSID | Unique ID of WiFi access point (MAC address of the hardware). |
| freq | The frequency on which the access point operates. |
| level | The strength of the signal. |

> **Note**:

> * Epochtime format can be parsed with the Pandas, "to_datetime" function as demonstrated in M1_NB2, section 1.1.

The first example will only look at the BSSID, while subsequent examples will also look at the timestamp. Students who have worked with BSSIDs previously will notice the lack of an SSID, the network name. This was removed by the Dartmouth researchers prior to the release of the dataset due to institutional security concerns. While it could be argued that this is one of the most useful pieces of information, our analysis does not require this feature.

#### 1.3 Check for missing values
You can use the Pandas "count" method to provide a quick overview of the entries in each column that contain values (non-empty). These entries can then be compared with the total number of rows in the dataset. 

In [4]:
print 'Number values (non-empty records) for each column:'
print user00.count()

print '\n'
print ('Overall number of rows:\n{}'.format(len(user00)))

Number values (non-empty records) for each column:
time     446110
BSSID    446110
freq     446110
level    446110
dtype: int64


Overall number of rows:
446110


Since the columns all contain 446110 records, there are no missing values. So far so good!

<br>
<div class="alert alert-info">
<b>Exercise 1 Start.</b>
</div>

### Instructions

> Can you recall where you would use the Pandas "info()" and "describe()" functions from Module 1 notebook 2? Use the Pandas libraries to provide the output for each of the functions in the provided cells below. The syntax should be:

>   `dataset.function()`

> where "dataset" is the dataset you are analysing (user00 in this case), and "function" is the method or function you wish to apply (either "info" or "describe").

In [5]:
# Your answer here. (Pandas info)
user00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446110 entries, 0 to 446109
Data columns (total 4 columns):
time     446110 non-null int64
BSSID    446110 non-null object
freq     446110 non-null int64
level    446110 non-null int64
dtypes: int64(3), object(1)
memory usage: 13.6+ MB


In [6]:
# Your answer here. (Pandas describe)
user00.describe()

,time,freq,level
count,4.461100e+05,446110.000000,446110.000000
mean,1.367449e+09,2767.051106,-80.725689
std,1.372051e+06,947.090139,15.926016
min,1.364357e+09,2412.000000,-99.000000
25%,1.366571e+09,2422.000000,-90.000000
50%,1.367696e+09,2437.000000,-85.000000
75%,1.368028e+09,2462.000000,-77.000000
max,1.370051e+09,5825.000000,0.000000


<br>
<div class="alert alert-info">
<b>Exercise 1 End.</b>
</div>

> **Exercise complete**:
    
> This is a good time to "Save and Checkpoint".

#### 1.4 Data validation
As you can see from the first few lines of the data, the epochtime format is not very useful when trying to review datetimes. In the cell below we use the Pandas "to_datetime()" function to create a new variable, called readable_time, which has the epochtime converted into something humans can understand.
Panda's default output from to_datetime is in units of milliseconds, which we wish to change to seconds. To use seconds instead, we supply the optional argument, "unit='s' ", to produce the desired output.

In [7]:
# Repeat the function from section 1.1 to review the contents of the "time" column.
user00.head(3)

,time,BSSID,freq,level
0,1364356963,00:60:b3:07:1d:27,2427,-90
1,1364356963,f8:1e:df:fd:4a:4b,2417,-64
2,1364358163,f8:1e:df:fd:4a:4b,2417,-69


In [8]:
readable_time = pd.to_datetime(user00.time, unit='s')

You can use the 'print' command to display the maximum and minimum times in our new dataset. Notice that by adding .min() or .max() after readable_time, Python will apply the method to find these values, and print them in place of {}, which we use for string formatting. The .format function takes as many arguments as there are {}'s, so in this case, 2.

In [9]:
# Manual review.
print 'Existing times range between: {} and {}'.format(readable_time.min(), readable_time.max())

Existing times range between: 2013-03-27 04:02:43 and 2013-06-01 01:40:26


In [10]:
# Pandas describe function.
readable_time.describe()

count                  446110
unique                  61119
top       2013-05-25 16:09:08
freq                       57
first     2013-03-27 04:02:43
last      2013-06-01 01:40:26
Name: time, dtype: object

Next, we use the Pandas value_counts method to find the counts of unique values for observed frequencies in our converted dataset, readable_time. The full syntax for the value_counts method is:
> `series.value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)`

In [11]:
# Use the pandas value_count method to review observed frequencies.
f_counts = pd.value_counts(user00['freq'])
f_counts.head(10)

2462    114880
2437    100319
2412     86379
2417     24197
2457     11706
2427     10937
2422     10203
2447      9763
2442      8370
5180      7822
Name: freq, dtype: int64

You can review the other columns in the dataset, and explore any of the other features, based on the information you have been provided with.

<br>
<div class="alert alert-info">
<b>Exercise 2 Start.</b>
</div>

### Instructions
> Assume for the sake of the exercise, that u00 refers to a dataset created based on your activities.

> 1. Use the Pandas “value_count” method, demonstrated in section 1.4 with observed frequencies, to review the observations per BSSID ("user00['BSSID']"), and indicate which access point corresponds to your home location.

> 2. Add a comment to provide your justification for this choice of access point.

> 3. Provide a brief description of one or two other use cases where your justification in question 2 would be invalid. Think about the locations where you spend most of your time and what other kinds of behaviors you would expect in a large-scale experiment.

In [14]:
# Your answer here.
BSSID_counts = pd.value_counts(user00['BSSID'])
BSSID_counts.head(10)

f8:1e:df:fd:4a:4b    12996
00:26:b8:b7:41:85     2901
00:27:22:98:9e:2d     2243
00:26:18:9f:c0:c9     1898
06:26:bb:77:63:b7     1896
00:26:bb:77:63:b7     1801
00:0e:38:a4:b7:52     1740
00:13:19:8c:5c:c0     1738
00:24:89:0c:bf:23     1694
00:24:89:27:2c:b3     1668
Name: BSSID, dtype: int64

The access point in the first line should correspond to my home location.

Because the dtype number is much larger than the following numbers. The first two letterso f BSSID is totally different from the following BSSIDs. Furthermore, it is reasonable to see the first line of BSSID indicate the home location of the user.

The case can be that I am walking through several different free Wi-Fi spots. I might connect to each of them when I pass by them, then the BSSID indicated would be invalid. The locations I spend most of my time is my home and my study place. The other kinds of behaviors I would expect in a large-scale experiment, if the experiment lasts for long time, changing home address and moving the house would be reasonable behaviors. Besides, change of study place, change of work place would all be reasonable if the experiment lasts for enough time.


<br>
<div class="alert alert-info">
<b>Exercise 2 End.</b>
</div>

> **Exercise complete**:
    
> This is a good time to "Save and Checkpoint".

## 2. Submit your notebook

Please make sure that you:
- Perform a final "Save and Checkpoint";
- Download a copy of the notebook in ".ipynb" format to your local machine using "File", "Download as", and "IPython Notebook (.ipynb)", and
- Submit a copy of this file to the online campus.